In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import zipfile
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Preprocessing

Before we can commence any form of Deep Learning, we will have to sort the images into training and testing folders. This is done in order to faciliate the ImageDataGenerator from Keras.

In [ ]:
root = '/kaggle/input/dogs-vs-cats'
root_temp = '/kaggle/working/'

In [ ]:
#Removes train and test folders
try:
    os.rmdir(os.path.join(root_temp,'train'))
    os.rmdir(os.path.join(root_temp,'test1'))
    os.rmdir(os.path.join(root_temp,'val'))
except OSError as error:
    pass

In [ ]:
#Unzips files
with zipfile.ZipFile(os.path.join(root,'train.zip'), 'r') as zip_ref:
    zip_ref.extractall(os.path.join(root_temp))
with zipfile.ZipFile(os.path.join(root,'test1.zip'), 'r') as zip_ref:
    zip_ref.extractall(os.path.join(root_temp))

In [ ]:
#Confirm number of images
print('total number in training folder ', len(os.listdir(os.path.join(root_temp,'train'))))
print('total number in test folder ', len(os.listdir(os.path.join(root_temp,'test1'))))

In [ ]:
#Confirm images in
train_images = [fname for fname in os.listdir(os.path.join(root_temp,'train'))]
test_images = [fname for fname in os.listdir(os.path.join(root_temp,'test1'))]

In [ ]:
print('total number in training set ', len(train_images))
print('total number in test set ', len(test_images))

In [ ]:
pd.DataFrame(train_images).head()

In [ ]:
pd.DataFrame(test_images).head()

In [ ]:
train_cat = []
train_dog = []
for image in train_images:
    if 'cat' in image:
        train_cat.append(image)
    else: train_dog.append(image)

In [ ]:
print('total number of cats in training set ', len(train_cat))
print('total number of dogs in training set ', len(train_dog))
print('total number of images in test set ', len(test_images))

In [ ]:
try:
    os.mkdir(os.path.join(root_temp,'train','cat'))
    os.mkdir(os.path.join(root_temp,'train','dog'))
except OSError:
    pass

In [ ]:
import shutil 

for image in train_cat:
    shutil.move(os.path.join(root_temp,'train',image),os.path.join(root_temp,'train','cat',image))
    
for image in train_dog:
    shutil.move(os.path.join(root_temp,'train',image),os.path.join(root_temp,'train','dog',image))

root_train = os.path.join(root_temp,'train') 
train_cat = os.path.join(root_train,'cat')
train_dog = os.path.join(root_train,'dog')

root_val = os.path.join(root_temp,'val')
val_cat = os.path.join(root_val,'cat')
val_dog = os.path.join(root_val,'dog')

root_test = os.path.join(root_temp,'test1')

try:
    os.mkdir(root_val)
    os.mkdir(val_cat)
    os.mkdir(val_dog)
except OSError:
    pass

In [ ]:
for index in range(1500):
    #moves first 1500 images from /kaggle/working/train/cat to /kaggle/working/val/cat 
    shutil.move(os.path.join(root_temp,'train','cat',os.listdir(os.path.join(root_temp,'train','cat'))[index]),val_cat)
    shutil.move(os.path.join(root_temp,'train','dog',os.listdir(os.path.join(root_temp,'train','dog'))[index]),val_dog)

In [ ]:
#confirms the number of cats and dogs in folders
print('total number of cats in training folder ', len(os.listdir(train_cat)))
print('total number of dogs in training folder ', len(os.listdir(train_dog)))
print('total number of cats in validation folder ', len(os.listdir(val_cat)))
print('total number of dogs in validation folder ', len(os.listdir(val_dog)))
print('total number of iamges in testing folder ', len(os.listdir(root_test)))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale = (1./255))
test_gen = ImageDataGenerator(rescale = (1./255))

In [ ]:
os.listdir(root_test)

In [ ]:
len(os.listdir(root_test))

In [ ]:
test_dir = os.path.join(root_test,'test')
try :
    os.mkdir(test_dir)
except: OSError
pass

from pathlib import Path

#root test = /kaggle/working/test1
#test_dir = /kaggle/working/test1/test
for file in os.listdir(root_test):
    if 'test' not in file:
        shutil.move(root_test+'/'+file, test_dir+'/'+file)

In [ ]:
len(os.listdir(os.path.join(root_test,'test')))

In [ ]:
train_flow = train_gen.flow_from_directory(root_train, 
                                           target_size = (150,150),
                                          batch_size=25,
                                          class_mode = 'binary')
val_flow = test_gen.flow_from_directory(root_val, 
                                        target_size = (150,150),
                                        batch_size=25,
                                        class_mode = 'binary')
test_flow = test_gen.flow_from_directory(root_test, 
                                        target_size = (150,150),
                                        batch_size=25,
                                        class_mode = 'binary',
                                        shuffle = False)

In [ ]:
desired_accuracy = 0.95

class my_callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, log ={}):
        if (log.get('accuracy')>desired_accuracy):
            print('\Accuracy of {} reached, stopping training '.format(desired_accuracy))
            self.model.stop_training = True

stop_training_callback = my_callback

In [ ]:
# Modelling Time

basic_cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
basic_cnn.summary()

In [ ]:
basic_cnn.compile(loss = 'binary_crossentropy', metrics =['acc'], optimizer = tf.keras.optimizers.RMSprop(learning_rate = 1e-4))

basic_cnn_history = basic_cnn.fit_generator(train_flow,
                   validation_data = val_flow,
                   steps_per_epoch = 100,
                   epochs = 50,
                   validation_steps = 50)

In [ ]:
def plot_results(model_history):
    acc = model_history.history['acc']
    val_acc = model_history.history['val_acc']
    loss = model_history.history['loss']
    val_loss = model_history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')

    plt.figure()

    plt.plot(epochs, loss, 'r', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [ ]:
plot_results(basic_cnn_history)

In [ ]:
# Transfer Learning
pre_trainned_model = tf.keras.applications.InceptionV3(include_top = False,
                                                       weights = 'imagenet',
                                                       input_shape = (150,150,3))

for layer in pre_trainned_model.layers:
    layer.trainable = False

In [ ]:
pre_trainned_model.summary()

In [ ]:
last_layer = pre_trainned_model.get_layer('mixed9_1')

pretrainned_last_output = last_layer.output

x = tf.keras.layers.Flatten()(pretrainned_last_output)
x = tf.keras.layers.Dense(512, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

new_model = tf.keras.models.Model(pre_trainned_model.input , x)

new_model.summary()

In [ ]:
desired_accuracy = 0.90
class my_callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, log={}):
        if (log.get('val_acc')>desired_accuracy):
            print('\nDesired validation accuracy reached, stop training')
            self.model.stop_training = True
callbacks = my_callback

In [ ]:
new_model.compile(loss='binary_crossentropy', metrics = ['acc'], optimizer = tf.keras.optimizers.RMSprop(learning_rate = 1e-4))
new_model_history = new_model.fit_generator(train_flow,
                                           validation_data = val_flow,
                                           steps_per_epoch = 100,
                                           epochs = 5,
                                           validation_steps = 50,
#                                            callbacks = [callbacks]
                                           )

In [ ]:
new_model_history.history.keys()

In [ ]:
acc = new_model_history.history['acc']
val_acc = new_model_history.history['val_acc']
loss = new_model_history.history['loss']
val_loss = new_model_history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
last_layer = pre_trainned_model.get_layer('mixed7')

pretrainned_last_output = last_layer.output

x = tf.keras.layers.Flatten()(pretrainned_last_output)
x = tf.keras.layers.Dense(512, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

new_model_mixed7 = tf.keras.models.Model(pre_trainned_model.input , x)

new_model_mixed7.compile(loss='binary_crossentropy', metrics = ['acc'], optimizer = tf.keras.optimizers.RMSprop(learning_rate = 1e-4))
new_model_mixed7_history = new_model.fit_generator(train_flow,
                                                    validation_data = val_flow,
                                                    steps_per_epoch = 100,
                                                    epochs = 5,
                                                    validation_steps = 50,
                                                  #callbacks = [callbacks]
                                                  )

In [ ]:
plot_results(new_model_mixed7_history)

In [ ]:
last_layer = pre_trainned_model.get_layer('mixed1')

pretrainned_last_output = last_layer.output

x = tf.keras.layers.Flatten()(pretrainned_last_output)
x = tf.keras.layers.Dense(256, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

new_model_mixed1 = tf.keras.models.Model(pre_trainned_model.input , x)

new_model_mixed1.compile(loss='binary_crossentropy', metrics = ['acc'], optimizer = tf.keras.optimizers.RMSprop(learning_rate = 1e-4))
new_model_mixed1_history = new_model.fit_generator(train_flow,
                                                   validation_data = val_flow,
                                                   steps_per_epoch = 100,
                                                   epochs = 5,
                                                   validation_steps = 50,
                                                  #callbacks = [callbacks]
                                                  )

In [ ]:
#Lets try another application namely VGG
from tensorflow.keras.applications import VGG16

In [ ]:
pretrain_vgg = VGG16(include_top = False,
                    input_shape = (150,150,3),
                    weights = 'imagenet')


In [ ]:
for layer in pretrain_vgg.layers:
    layer.trainable = False

pretrain_vgg.summary()

In [ ]:
pretrain_vgg_output =  pretrain_vgg.get_layer('block5_pool').output 

x = tf.keras.layers.Flatten()(pretrain_vgg_output)
x = tf.keras.layers.Dense(512,activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1,activation = 'sigmoid')(x)

pretrain_vgg_model = tf.keras.models.Model(pretrain_vgg.input, x)
pretrain_vgg_model.compile(loss = 'binary_crossentropy',
                          metrics=['acc'],
                          optimizer = tf.keras.optimizers.RMSprop(lr=0.00001))
pretrain_vgg_history = pretrain_vgg_model.fit_generator(train_flow,
                                                       validation_data = val_flow,
                                                       steps_per_epoch = 100,
                                                       epochs = 10,
                                                       validation_steps = 50)

In [ ]:
plot_results(pretrain_vgg_history)

In [ ]:
len(root_test)

In [ ]:
test_flow

In [ ]:
predictions = pretrain_vgg_model.predict(test_flow)

In [ ]:
len(predictions)

In [ ]:
predictions = [1 if pred > 0.5 else 0 for pred in predictions]

In [ ]:
predictions[0]

In [ ]:
#label_ids = test_flow.filenames

In [ ]:
label_ids = [fname for fname in os.listdir(test_dir)]

In [ ]:
label_ids[0]

In [ ]:
import csv
data = zip(*(label_ids, predictions))

df = pd.DataFrame(data, columns = ['id' , 'label'])

In [ ]:
df = df.set_index('id')

In [ ]:
df.head()

In [ ]:
df.to_csv('submission.csv', index=False)